# Importing Relevant Python Libraries

In [63]:
# Data Analysis and Wrangling
import pandas as pd
import numpy as np
import random as rnd
import math

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Machine Learning
from sklearn import linear_model
from sklearn.model_selection import train_test_split

# Extracting Data

In [180]:
list1 = pd.read_csv('listings.csv')
list2 = pd.read_csv('listings2.csv')

# Data Size Metrics

In [181]:
#print(list1.columns.values)
print(list1.shape)

(3818, 16)


In [182]:
#list2.columns.values)
print(list2.shape)

(3818, 92)


In [183]:
# Join list1 and list2 to create the x vector
X = pd.concat([list1, list2], axis=1)
X.shape # Original Data Frame

(3818, 108)

In [185]:
X['Sec_Dep'] = X['security_deposit'].replace(np.nan, '$0', regex=True)

In [190]:
X.drop(['security_deposit'], axis = 1, inplace = True)
X.shape

(3818, 108)

In [193]:
X['security_deposit']=X['Sec_Dep'].replace('[\$,]', '', regex=True).astype(float)

In [195]:
X.drop(['Sec_Dep'], axis = 1, inplace = True)
X.shape

(3818, 108)

# Feature Engineering

### Property Type

In [197]:
X=X[X['property_type'].notnull()]
X['property_type'].value_counts()

House              1733
Apartment          1708
Townhouse           118
Condominium          91
Loft                 40
Bed & Breakfast      37
Other                22
Cabin                21
Camper/RV            13
Bungalow             13
Boat                  8
Tent                  5
Treehouse             3
Dorm                  2
Chalet                2
Yurt                  1
Name: property_type, dtype: int64

In [198]:
X.shape

(3817, 108)

In [199]:
# Increasing the feature space
i=0
for item in X['property_type'].unique():
    a=X['property_type'] == X['property_type'].unique()[i]
    X[item] = a.map(lambda x: 1 if x == True else -1)
    i=i+1
X.shape

(3817, 124)

### Bed Type

In [200]:
X=X[X['bed_type'].notnull()]
X['bed_type'].unique()

array(['Real Bed', 'Futon', 'Pull-out Sofa', 'Airbed', 'Couch'], dtype=object)

In [201]:
# Increasing the feature space
i=0
for item in X['bed_type'].unique():
    a=X['bed_type'] == X['bed_type'].unique()[i]
    X[item] = a.map(lambda x: 1 if x == True else -1)
    i=i+1
X.shape

(3817, 129)

### Extracting Numerical Values

In [202]:
Z1=X.loc[:, X.dtypes == np.float64] #Extracting columns with values of type float 
Z2=X.loc[:, X.dtypes == np.int64]   #Extracting columns with values of type int
X_numeric=pd.concat([Z1,Z2], axis=1)

In [223]:
# X_numeric has duplicate columns. The code below removes the duplicate columns
_, i = np.unique(X_numeric.columns, return_index=True)
X_Num_Cov=X_numeric.iloc[:, i]
X_Num_Cov.shape
X_Num_Cov.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3817 entries, 0 to 3817
Data columns (total 53 columns):
Airbed                            3817 non-null int64
Apartment                         3817 non-null int64
Bed & Breakfast                   3817 non-null int64
Boat                              3817 non-null int64
Bungalow                          3817 non-null int64
Cabin                             3817 non-null int64
Camper/RV                         3817 non-null int64
Chalet                            3817 non-null int64
Condominium                       3817 non-null int64
Couch                             3817 non-null int64
Dorm                              3817 non-null int64
Futon                             3817 non-null int64
House                             3817 non-null int64
Loft                              3817 non-null int64
Other                             3817 non-null int64
Pull-out Sofa                     3817 non-null int64
Real Bed                     

### Latitude and Longitude

### Using KNN for local model fitting

In [204]:
# Defining the points for KNN algorithm
lat=np.linspace(47.55,47.70,3)
long=np.linspace(-122.3875,-122.2875,3)
encode=[]
for i in range(0,len(X_Num_Cov['latitude'])):
    delta_arr=[]
    for j in range(0,len(lat)):
        for k in range(0,len(long)):
            a=(X_Num_Cov['latitude'].iloc[i]-lat[j])**2
            b=(X_Num_Cov['longitude'].iloc[i]-long[k])**2
            c=a+b
            delta_arr.append(c)
    delta=min(delta_arr)
    encode.append((np.array(delta_arr)<=delta).astype(int))

In [205]:
knn1=[]
knn2=[]
knn3=[]
knn4=[]
knn5=[]
knn6=[]
knn7=[]
knn8=[]
knn9=[]
for i in range(0,len(encode)):
    knn1.append(encode[i][0])
    knn2.append(encode[i][1])
    knn3.append(encode[i][2])
    knn4.append(encode[i][3])
    knn5.append(encode[i][4])
    knn6.append(encode[i][5])
    knn7.append(encode[i][6])
    knn8.append(encode[i][7])
    knn9.append(encode[i][8])

In [206]:
knn1_s=pd.Series( (v for v in np.array(knn1)) ,name='knn1')
knn2_s=pd.Series( (v for v in np.array(knn2)),name='knn2' )
knn3_s=pd.Series( (v for v in np.array(knn3)),name='knn3' )
knn4_s=pd.Series( (v for v in np.array(knn4)) ,name='knn4')
knn5_s=pd.Series( (v for v in np.array(knn5)) ,name='knn5')
knn6_s=pd.Series( (v for v in np.array(knn6)) ,name='knn6')
knn7_s=pd.Series( (v for v in np.array(knn7)) ,name='knn7')
knn8_s=pd.Series( (v for v in np.array(knn8)) ,name='knn8')
knn9_s=pd.Series( (v for v in np.array(knn9)) ,name='knn9')
X_Num_Cov=pd.concat([X_Num_Cov,knn1_s,knn2_s,knn3_s,knn4_s,knn5_s,knn6_s,knn7_s,knn8_s,knn9_s],axis=1)
X_Num_Cov.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3818 entries, 0 to 3817
Data columns (total 62 columns):
Airbed                            3817 non-null float64
Apartment                         3817 non-null float64
Bed & Breakfast                   3817 non-null float64
Boat                              3817 non-null float64
Bungalow                          3817 non-null float64
Cabin                             3817 non-null float64
Camper/RV                         3817 non-null float64
Chalet                            3817 non-null float64
Condominium                       3817 non-null float64
Couch                             3817 non-null float64
Dorm                              3817 non-null float64
Futon                             3817 non-null float64
House                             3817 non-null float64
Loft                              3817 non-null float64
Other                             3817 non-null float64
Pull-out Sofa                     3817 non-null float

In [210]:
# This cell constructs the new feature space
space=['security_deposit','availability_30','accommodates','bathrooms','bedrooms','beds','price','reviews_per_month','review_scores_value','knn1',
      'knn2','knn3','knn4','knn5','knn6','knn7','knn8','knn9']
for item in X['property_type'].unique().tolist():
    space.append(item)
for item in X['bed_type'].unique().tolist():
    space.append(item)

In [211]:
# Selecting certian numerical features
X_select = X_Num_Cov[space]
# Filling 0 for missing values in reviews per month column
values={'reviews_per_month':0,'review_scores_value':0}
X_new=X_select.fillna(value=values) # New Dataframe with missing values substituted

In [212]:
X_new

,security_deposit,availability_30,accommodates,bathrooms,bedrooms,beds,price,reviews_per_month,review_scores_value,knn1,...,Dorm,Treehouse,Yurt,Chalet,Tent,Real Bed,Futon,Pull-out Sofa,Airbed,Couch
0,0.0,14.0,4.0,1.0,1.0,1.0,85.0,4.07,10.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0
1,100.0,13.0,4.0,1.0,1.0,1.0,150.0,1.48,10.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0
2,1000.0,1.0,11.0,4.5,5.0,7.0,975.0,1.15,10.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0
3,0.0,0.0,3.0,1.0,0.0,2.0,100.0,0.00,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0
4,700.0,30.0,6.0,2.0,3.0,3.0,450.0,0.89,9.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0
5,0.0,0.0,2.0,1.0,1.0,1.0,120.0,2.45,10.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0
6,150.0,20.0,2.0,1.0,1.0,1.0,80.0,2.46,10.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0
7,150.0,22.0,2.0,1.0,1.0,1.0,60.0,4.73,10.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0
8,0.0,6.0,2.0,1.0,1.0,1.0,90.0,1.22,10.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0
9,100.0,6.0,4.0,1.0,1.0,1.0,150.0,1.55,9.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0


In [213]:
# Dropping the missing values from the feature space
X_new = X_new.dropna()
# Creating the output space
Y = X_new['reviews_per_month']
X_new = X_new.drop(['reviews_per_month'], axis = 1)
print(X_new.info())
X_new.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3794 entries, 0 to 3816
Data columns (total 38 columns):
security_deposit       3794 non-null float64
availability_30        3794 non-null float64
accommodates           3794 non-null float64
bathrooms              3794 non-null float64
bedrooms               3794 non-null float64
beds                   3794 non-null float64
price                  3794 non-null float64
review_scores_value    3794 non-null float64
knn1                   3794 non-null float64
knn2                   3794 non-null float64
knn3                   3794 non-null float64
knn4                   3794 non-null float64
knn5                   3794 non-null float64
knn6                   3794 non-null float64
knn7                   3794 non-null float64
knn8                   3794 non-null float64
knn9                   3794 non-null float64
Apartment              3794 non-null float64
House                  3794 non-null float64
Cabin                  3794 non-null f

,security_deposit,availability_30,accommodates,bathrooms,bedrooms,beds,price,review_scores_value,knn1,knn2,...,Dorm,Treehouse,Yurt,Chalet,Tent,Real Bed,Futon,Pull-out Sofa,Airbed,Couch
0,0.0,14.0,4.0,1.0,1.0,1.0,85.0,10.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0
1,100.0,13.0,4.0,1.0,1.0,1.0,150.0,10.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0
2,1000.0,1.0,11.0,4.5,5.0,7.0,975.0,10.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0
3,0.0,0.0,3.0,1.0,0.0,2.0,100.0,0.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0
4,700.0,30.0,6.0,2.0,3.0,3.0,450.0,9.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,-1.0


# Preliminary Model Analysis

### Splitting data into training and testing sets

In [214]:
from sklearn.model_selection import cross_val_score
num_test = 0.20 #20% of the data set will be used for testing the final model
X_train, X_test, y_train, y_test = train_test_split(X_new, Y, test_size=num_test, random_state=100)

## Ridge regression + Hyperparameter Tuning via Cross Validation

In [215]:
from sklearn.linear_model import Ridge
clf = Ridge(alpha=5.0)
scores = cross_val_score(clf, X_train, y_train, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.24 (+/- 0.04)


#### Fitting the selected model 

In [216]:
clf.fit(X_train,y_train)

Ridge(alpha=5.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

#### Making Predictions

In [217]:
y_test_predict=clf.predict(X_test)

In [218]:
y_train_predict=clf.predict(X_train)

#### Model Performance Evaluation

In [219]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms_test = sqrt(mean_squared_error(y_test, y_test_predict))
rms_test

1.5728077534861236

In [220]:
rms_train = sqrt(mean_squared_error(y_train, y_train_predict))
rms_train

1.5847620822978457

In [221]:
clf.score(X_train, y_train)

0.26136137839092044

In [222]:
clf.score(X_test, y_test)

0.24113477454384458